In [7]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import xgboost as xgb
import joblib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime
import warnings
warnings.filterwarnings("ignore") 
plt.rc('font', family='NanumGothic') 
#print (plt.rcParams['font.family'] )

In [13]:
df_B = pd.read_csv('시간별_발생건수_B.csv')
df_C = pd.read_csv('시간별_발생건수_C.csv')
df_BC = pd.concat([df_B, df_C])


df_BC = df_BC.drop(['Unnamed: 0','월', '일', '시', '지역'],axis=1)
df_BC = df_BC.reset_index(drop=True)


# datetime으로 변환
df_BC['일시']= pd.to_datetime(df_BC['일시'])
df_BC['DATE'] = df_BC['일시'].dt.strftime('%Y-%m-%d')

df_BC = df_BC.drop('일시', axis=1)
user = input('ex. 20211205 *12월25일 이전만 입력 : ')

year = user[:4]
month = user[4:6]
days = user[6:]

date = year + '-' + month + '-' + days
date = pd.date_range(user, periods=7, freq='d')
date = [i.strftime('%Y-%m-%d') for i in date]

# target 0,1 변경
df_BC['발생건수'] = [1 if i>0 else 0 for i in df_BC['발생건수']]

# train
train = df_BC.loc[~df_BC['DATE'].isin(date)]
train = train.reset_index(drop=True)

# test
test = df_BC.loc[df_BC['DATE'].isin(date)]
test_list = []

for _,row in test.groupby('DATE'):
    row = row.reset_index(drop=True)
    row = row.sample(n=1)
    test_list.append(row)
    
test = pd.concat(test_list)
test = test.reset_index(drop=True)


X_train = train.drop(['DATE', '발생건수'], axis=1)
Y_train = train[['발생건수']]

X_test = test.drop(['DATE', '발생건수'], axis=1)
Y_test = test[['발생건수']]

def onehot_encoder(train, test, columns):
    target_list = set(train[columns])
    
    for i in target_list:
        train[f'{columns}_{i}'] = train[columns] == i
        test[f'{columns}_{i}'] = test[columns] == i
        train[f'{columns}_{i}'] = train[f'{columns}_{i}'].astype('int')
        test[f'{columns}_{i}'] = test[f'{columns}_{i}'].astype('int')
    
    train = train.drop(columns, axis=1)
    test = test.drop(columns, axis=1)
    
    return train, test


# 방향 원-핫 원코딩
X_train,X_test = onehot_encoder(X_train,X_test,'방향')
# 정규화 
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# 스모트
smote = SMOTE(random_state=0)
X_train_over, Y_train_over = smote.fit_resample(X_train, Y_train) 

print('SMOTE 적용 전 Train용 피처/레이블 데이터 세트: ', X_train.shape, Y_train.shape)
print('SMOTE 적용 후 Train 피처/레이블 데이터 세트: ', X_train_over.shape, Y_train_over.shape)
print('SMOTE 적용 후 Labels 값 분포: \n', Y_train_over.value_counts())
# modeling
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train_over, Y_train_over)
y_pred = rfc.predict(X_test)
from sklearn.metrics import accuracy_score
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))
out = y_pred
out = y_pred
for i in range(0,7,1):
    print(date[i],'의 민원발생 여부는', out[i],'입니다.')

ex. 20211205 *12월25일 이전만 입력 :  20210101


SMOTE 적용 전 Train용 피처/레이블 데이터 세트:  (17204, 21) (17204, 1)
SMOTE 적용 후 Train 피처/레이블 데이터 세트:  (34084, 21) (34084, 1)
SMOTE 적용 후 Labels 값 분포: 
 발생건수
0       17042
1       17042
dtype: int64
Model accuracy score with 10 decision-trees : 1.0000
2021-01-01 의 민원발생 여부는 0 입니다.
2021-01-02 의 민원발생 여부는 0 입니다.
2021-01-03 의 민원발생 여부는 0 입니다.
2021-01-04 의 민원발생 여부는 0 입니다.
2021-01-05 의 민원발생 여부는 0 입니다.
2021-01-06 의 민원발생 여부는 0 입니다.
2021-01-07 의 민원발생 여부는 0 입니다.


In [47]:
# modeling
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train_over, Y_train_over)
y_pred = rfc.predict(X_test)

/tmp/ipykernel_92827/1333567772.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train_over, Y_train_over)


In [48]:
from sklearn.metrics import accuracy_score
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.9965


In [34]:
rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)
# fit the model to the training set
rfc_100.fit(X_train, Y_train)
y_pred_100 = rfc_100.predict(X_test)
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(Y_test, y_pred_100)))

/tmp/ipykernel_92827/2551999779.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc_100.fit(X_train, Y_train)


Model accuracy score with 100 decision-trees : 0.9646


In [35]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rfc.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index = X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature Importances')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [263]:
target_df = target_df.drop(['일시', 'DATE'], axis=1)